In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv("data/TAQ_15Min_AAPL_2023.csv")
df.index = df.datetime
df.info()

In [ ]:
def predict_volumes(mu, Sigma, true_volumes):
    """
    param mu: means
    param Sigma: covariance
    param true_volumes: true volumes, i.e., means[0] predicts true_volumes[0]
    """
    n= mu.shape[0]
    
    predictions = np.zeros(n)
    predictions[0] = mu[0]
    
    for i in range(1,n):
        ### following Wikipedia's notation, we observe x2 and update x1
        
        mu1 = mu[i:]
        mu2 = mu[:i]
        a = true_volumes[:i]
        
        mu_bar = mu[i:]
        Sigma_12 = Sigma[i:, :i]
        Sigma_22 = Sigma[:i, :i]
        Sigma_21 = Sigma[:i, i:]
#         assert Sigma_21 == Sigma_12.T

        mu_bar = mu1 + Sigma_12 @ np.linalg.inv(Sigma_22) @ (a-mu2)
        
        predictions[i] = mu_bar[0]
    
    return predictions
    


In [ ]:
def lag_col(pd_col, i):
    d = {"VOLUME" : "VOLUME_{}".format(10-i)}
    pd_col.rename(columns=d, inplace=True)
    return pd_col.shift(-i)[:-9]

X1 = pd.concat([lag_col(pd.DataFrame(df["VOLUME"]), i) for i in range(10)], axis=1)
times = X1.index
X = X1.to_numpy()

In [ ]:
np.random.seed(0)

A = np.random.normal(size=(10,10))

Sigma = A@A.T

#X = np.random.multivariate_normal(np.zeros(10),Sigma, 1000)

mu = X.mean(axis=0)

benchmark_error = ((X-mu)**2).mean()
# so we would do 9 updates per row since we have 10 columns 

mu = X.mean(axis=0)
Sigma = np.cov(X[:-1].T)
true_volumes = X[-1]

pred = predict_volumes(mu, Sigma, true_volumes)
print(f"MSE without mean update: {((mu-true_volumes)**2).mean()}")
print(f"MSE with mean update: {((pred-true_volumes)**2).mean()}")

In [ ]:
plt.semilogy(range(10), pred, label="Predicted", linestyle="dashed")
plt.semilogy(range(-10,1), X1["VOLUME_1"][-20:-9], label="Last 10 of data", color="black")
plt.semilogy(range(10), true_volumes, label="True", color="blue")
plt.legend()